https://github.com/haotian-liu/LLaVA \
https://github.com/haotian-liu/LLaVA/blob/main/docs/MODEL_ZOO.md

In [1]:
import os
import pandas as pd
import json

#### Reading the Pathology Test Data

In [2]:
questions = pd.read_csv("../pathology_test_data/questions.csv")
questions

,image_name,question,original_responses
0,ck_PTGC_2x.jpeg,How can you best describe the low-power patter...,The lymph node shows a mixed follicular and in...
1,ck_PTGC_2x.jpeg,What are three main differential diagnostic co...,"Based on the low-power pattern, the primary co..."
2,ck_PTGC_5x.jpeg,What is the morphologic alteration being depic...,"The image shows a central enlarged, somewhat i..."
3,ck_PTGC_5x.jpeg,What is the expected immunoarchitecture of the...,This image shows an enlarged secondary follicl...
4,ck_serositis_4x.jpg,What is the most common source for the change ...,"The changes here show extensive serositis, wit..."
5,ck_serositis_4x.jpg,What is the specific anatomic region shown in ...,The right half shows the muscularis propria an...
6,ck_steatohepatitis_100x.jpg,If these histologic changes included conspicuo...,There is fatty liver disease and if concurrent...
7,ck_steatohepatitis_100x.jpg,What would the primary histologic feature to s...,Wilson's disease has many non-specific finding...
8,ck_steatohepatitis_200x.jpg,In an overweight adolescent with mildly increa...,"In the background of steatosis, there is incre..."


In [3]:
images_path = os.path.join(os.getcwd(), "../pathology_test_data/images")
images_list = os.listdir(images_path)
images_list

['ck_steatohepatitis_100x.jpg',
 'ck_PTGC_2x.jpeg',
 'ck_serositis_4x.jpg',
 'ck_steatohepatitis_200x.jpg',
 'ck_PTGC_5x.jpeg']

#### Creating a message structure

In [7]:
messages = []
for index, row in questions.iterrows():
    image_name = row["image_name"]
    question = row["question"]
    question_id = index
    
    message = {
                "question_id": question_id,
                "image": image_name,
                "text": question,
              }
    
    messages.append(message)

#### Creating a jsonl file with each line mentioning a specific question/image

In [8]:
os.makedirs("./temp_files", exist_ok=True)
file_name = "llava_1.6_7b.jsonl"
file_path = os.path.join("./temp_files", file_name)
with open(file_path, "w") as f:
    for item in messages:
        f.write(json.dumps(item) + "\n")

#### Running the inference code from the command line

In [9]:
!export PYTHONPATH=/data/mn27889/path-open-data/LLaVA:$PYTHONPATH

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python llava/eval/model_vqa.py \
    --model-path liuhaotian/llava-v1.6-vicuna-7b \
    --image-folder /data/mn27889/path-open-data/pathology_test_data/images \
    --question-file /data/mn27889/path-open-data/vlm_pathology_test_data_responses/temp_files/llava_1.6_7b.jsonl \
    --answers-file /data/mn27889/path-open-data/vlm_pathology_test_data_responses/temp_files/llava_1.6_7b_resp.jsonl

Traceback (most recent call last):
  File "/data/mn27889/path-open-data/vlm_pathology_test_data_responses/../LLaVA/llava/eval/model_vqa.py", line 8, in <module>
    from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
ModuleNotFoundError: No module named 'llava'


#### Reading the Responses file

In [16]:
resp_file_name = "llava_1.6_7b_resp.jsonl"
resp_file_path = os.path.join("./temp_files", resp_file_name)

indices = []
llava_responses = []
with open(resp_file_path, "r") as f:
    for line in f:
        json_data = json.loads(line)
        indices.append(json_data["question_id"])
        llava_responses.append(json_data["text"])

In [17]:
responses = pd.Series(llava_responses, index=indices)
responses

0    The image you've provided appears to be a micr...
1    The image you've provided appears to show a ly...
2    The image you've provided appears to show an e...
3    The image you've provided appears to show a hi...
4    The image shows a histological slide of tissue...
5    The image on the right half of the image appea...
6    The histologic changes shown in the image incl...
7    Wilson's disease is a rare genetic disorder ch...
8    The image you've provided appears to be a micr...
dtype: object

In [20]:
os.makedirs("./responses", exist_ok=True)
questions['llava_1.6_7b_response'] = responses
questions.to_csv("./responses/llava_1.6_7b_responses.csv", index=False)